In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf


PLOTS_DIR = f"plots/assignment_rnn/"
MODEL_DIR = f"models/assignment_rnn_focal_loss"
#MODEL_DIR = f"../models/FeatureConcatTransformer_HLF_d256_l10_h8"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)

load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(data_configs["data_path"]["nominal"], max_events=1_000_000)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.5)
del DataProcessor

2025-11-28 10:33:20.488284: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764322400.512938 2581029 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764322400.520999 2581029 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764322400.540335 2581029 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764322400.540366 2581029 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764322400.540368 2581029 computation_placer.cc:177] computation placer alr

In [ ]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatRNN(data_config, name=r"RNN+ $\nu^2$-Flows")

#TransformerMatcher.load_model(f"{MODEL_DIR}/model.keras")
TransformerMatcher.build_model(
    hidden_dim=32,
    num_layers=6,
    dropout_rate=0.1,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.FocalAssignmentLoss(focal_gamma=3.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatRNN is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764322479.170990 2581029 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1,
        ),
    ]
)

Epoch 1/100


I0000 00:00:1764322499.739965 2582983 service.cc:152] XLA service 0x7f7d8800b0b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764322499.739995 2582983 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-28 10:35:00.483723: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-28 10:35:00.900043: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764322503.945921 2582983 cuda_dnn.cc:529] Loaded cuDNN version 91500


  4/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.2109 - loss: 0.1711    

I0000 00:00:1764322516.565623 2582983 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


387/391 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1933 - loss: 0.1693

2025-11-28 10:35:23.883295: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert


391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1929 - loss: 0.1692

2025-11-28 10:35:40.432976: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-28 10:35:43.734210: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatRNN_1/met_input_transform_1/assert_equal_1/Assert/Assert


391/391 ━━━━━━━━━━━━━━━━━━━━ 64s 76ms/step - accuracy: 0.1928 - loss: 0.1692 - val_accuracy: 0.1002 - val_loss: 0.1650 - learning_rate: 1.0000e-04
Epoch 2/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.1698 - loss: 0.1655 - val_accuracy: 0.2896 - val_loss: 0.1572 - learning_rate: 1.0000e-04
Epoch 3/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.2700 - loss: 0.1595 - val_accuracy: 0.3459 - val_loss: 0.1458 - learning_rate: 1.0000e-04
Epoch 4/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.3054 - loss: 0.1495 - val_accuracy: 0.4286 - val_loss: 0.1239 - learning_rate: 1.0000e-04
Epoch 5/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.4227 - loss: 0.1315 - val_accuracy: 0.5518 - val_loss: 0.1129 - learning_rate: 1.0000e-04
Epoch 6/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.4966 - loss: 0.1227 - val_accuracy: 0.5822 - val_loss: 0.1071 - learning_rate: 1.0000e-04
Epoch 7/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accurac

In [4]:
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/assignment_rnn/model.keras
